imports

In [2]:
import csv
import pandas as pd
import numpy as np
import tensorflow as tf

inspecting the dataset

In [3]:
with open("data/bbc-text.csv", 'r') as csvfile:
    print(f"First line (header) looks like this:\n\n{csvfile.readline()}")
    print(f"Each data point looks like this:\n\n{csvfile.readline()}")

First line (header) looks like this:

category,text

Each data point looks like this:

tech,tv future in the hands of viewers with home theatre systems  plasma high-definition tvs  and digital video recorders moving into the living room  the way people watch tv will be radically different in five years  time.  that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes. with the us leading the trend  programmes and other content will be delivered to viewers via home networks  through cable  satellite  telecoms companies  and broadband service providers to front rooms and portable devices.  one of the most talked-about technologies of ces has been digital and personal video recorders (dvr and pvr). these set-top boxes  like the us s tivo and the uk s sky+ system  allow people to record  store  play  pause and forward wind tv programmes when they want.  essentially

parse data from file in form of lists -> sentences and labels

In [4]:
def parse_data_from_file(filename):

    sentences = []
    labels = []
    
    with open(filename, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)  
        for row in reader:
            sentences.append(row['text']) 
            labels.append(row['category'])   
            
    
    return sentences, labels

In [5]:
sentences, labels = parse_data_from_file("data/bbc-text.csv")

print(f"There are {len(sentences)} sentences in the dataset.\n")
print(f"First sentence has {len(sentences[0].split())} words.\n")
print(f"There are {len(labels)} labels in the dataset.\n")
print(f"The first 5 labels are {labels[:5]}\n\n")

There are 2225 sentences in the dataset.

First sentence has 737 words.

There are 2225 labels in the dataset.

The first 5 labels are ['tech', 'business', 'sport', 'sport', 'entertainment']




a list of common stopwords

In [6]:
STOPWORDS = ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

a function which receives a string and return another string that excludes all of the stopwords provided from it, as well as converting it to lower-case. 

In [7]:
def standardize_func(sentence):
    
    sentence = sentence.lower()
    
    words = sentence.split()
    
    filtered_words = [word for word in words if word not in STOPWORDS]
    
    sentence = " ".join(filtered_words)

    return sentence

In [8]:
test_sentence = "Hello! We're just about to see this function in action =)"
standardized_sentence = standardize_func(test_sentence)
print(f"Original sentence is:\n{test_sentence}\n\nAfter standardizing:\n{standardized_sentence}")

standard_sentences = [standardize_func(sentence) for sentence in sentences]

print("\n\n--- Apply the standardization to the dataset ---\n")
print(f"There are {len(standard_sentences)} sentences in the dataset.\n")
print(f"First sentence has {len(sentences[0].split())} words originally.\n")
print(f"First sentence has {len(standard_sentences[0].split())} words (after removing stopwords).\n")

Original sentence is:
Hello! We're just about to see this function in action =)

After standardizing:
hello! just see function action =)


--- Apply the standardization to the dataset ---

There are 2225 sentences in the dataset.

First sentence has 737 words originally.

First sentence has 436 words (after removing stopwords).



vectorizing the sentences of dataset

In [9]:
def fit_vectorizer(sentences):

    vectorizer = tf.keras.layers.TextVectorization()
    
    vectorizer.adapt(sentences)
    
    return vectorizer

In [10]:
vectorizer = fit_vectorizer(standard_sentences)

vocabulary = vectorizer.get_vocabulary()

print(f"Vocabulary contains {len(vocabulary)} words\n")
print("[UNK] token included in vocabulary" if "[UNK]" in vocabulary else "[UNK] token NOT included in vocabulary")

I0000 00:00:1755189442.347854   50462 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5065 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Vocabulary contains 33088 words

[UNK] token included in vocabulary


In [11]:
padded_sequences = vectorizer(standard_sentences)

print(f"First padded sequence looks like this: \n\n{padded_sequences[0]}\n")
print(f"Tensor of all sequences has shape: {padded_sequences.shape}\n")
print(f"This means there are {padded_sequences.shape[0]} sequences in total and each one has a size of {padded_sequences.shape[1]}")

First padded sequence looks like this: 

[  93  155 1186 ...    0    0    0]

Tensor of all sequences has shape: (2225, 2418)

This means there are 2225 sequences in total and each one has a size of 2418


With the sentences already vectorized it is time to encode the labels so they can also be fed into a neural network.

In [12]:
def fit_label_encoder(labels):

    label_encoder = tf.keras.layers.StringLookup(mask_token=None, oov_token=None,num_oov_indices=0)
    label_encoder.adapt(labels)
    
    return label_encoder

In [13]:
label_encoder = fit_label_encoder(labels)

vocabulary = label_encoder.get_vocabulary()

label_sequences = label_encoder(labels)

print(f"Vocabulary of labels looks like this: {vocabulary}\n")
print(f"First ten labels: {labels[:10]}\n")
print(f"First ten label sequences: {label_sequences[:10]}\n")

Vocabulary of labels looks like this: [np.str_('sport'), np.str_('business'), np.str_('politics'), np.str_('tech'), np.str_('entertainment')]

First ten labels: ['tech', 'business', 'sport', 'sport', 'entertainment', 'politics', 'politics', 'sport', 'sport', 'entertainment']

First ten label sequences: [3 1 0 0 4 2 2 0 0 4]

